In [64]:
import librosa
import numpy as np
import shutil
import os
import re
from joblib import Parallel, delayed
import multiprocessing 
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from scipy import stats
import sklearn.metrics as metrices
import seaborn as sns
import librosa.display
from sklearn.neighbors import KNeighborsClassifier as KNN

In [204]:
raw_dir='recordings/selected_recordings/'
silence_removed_dir='recordings/silence_removed/'
unified_dir='recordings/silence_removed - Copy/'

In [205]:
mfcc_window_size=0.03
mfcc_stride_size=0.007
mfcc_num_of_features=13

In [206]:
def extract_mfcc(fname):
    y,sr=librosa.load(unified_dir+fname)
    result=librosa.feature.mfcc(y=y,n_mfcc=mfcc_num_of_features,hop_length=int(mfcc_window_size*sr), n_fft=int(mfcc_stride_size*sr))
    #result=librosa.util.normalize(result)
    #Add label as well
    if 'mandarin' in fname:
        label=0
    elif 'english' in fname:
        label=1
    elif 'spanish' in fname:
        label=2
    else:
        label=-1
    return [result,label]

In [207]:
result=Parallel(n_jobs=12)(delayed(extract_mfcc)(fname) for fname in os.listdir(unified_dir))

In [208]:
X=[]
y=[]
for i in result:
    X.append(i[0].T)
    y.append(i[1])

In [209]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.1)

In [210]:
X_train[0].shape

(619, 13)

In [211]:
xr=X_train[0].shape[0]

In [212]:
def change_input_format(X):
    Xc=np.stack(X,axis=0)
#     Xc=np.swapaxes(Xc,0,1)
#     Xc=np.swapaxes(Xc,1,2)
    return Xc

In [213]:
X_train=change_input_format(X_train)
X_test=change_input_format(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

ValueError: all input arrays must have the same shape

In [195]:
y_test.shape

(20,)

In [196]:
knn_X_train=np.concatenate(X_train,axis=0)
knn_X_test=np.concatenate(X_test,axis=0)
knn_y_train=[]
knn_y_test=[]
for i in y_train:
    for j in range(xr):
        knn_y_train.append(i)
for i in y_test:
    for j in range(xr):
        knn_y_test.append(i)
knn_y_train=np.array(knn_y_train)
knn_y_test=np.array(knn_y_test)

In [197]:
knnm=KNN()
knnm.fit(knn_X_train,knn_y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_p=[]
for i in range(20):
    single_result=[]
    for j in range(i*xr,(i+1)*xr):
        single_result.append(knnm.predict(knn_X_test[j,:].reshape(1, -1)))
    y_p.append(stats.mode(np.array(single_result))[0][0][0])

In [198]:
y_p=[]
for i in range(20):
    sre=knnm.predict(knn_X_test[i*xr:(i+1)*xr,:])
    y_p.append(stats.mode(sre)[0][0])

In [202]:
20*xr

13340

In [203]:
knn_X_test.shape

(13340, 13)

In [199]:
y_p

[2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 2, 2, 1]

In [200]:
y_test

array([0, 2, 1, 0, 0, 1, 0, 2, 0, 1, 2, 1, 2, 2, 1, 0, 2, 2, 0, 1])

In [201]:
metrices.accuracy_score(np.array(y_p),y_test)

0.5